In [1]:
import torch
import numpy as np
import pandas as pd
import os
import sys
import json

sys.path.append(os.path.join(sys.path[0], '../'))
from models import manager_for_sagittarius
from evaluation import initialize_experiment
from LINCS import utils

In [2]:
device = 'cuda:1'

# Load the data

In [3]:
initialize_experiment.initialize_random_seed(0)

dl = utils.load_all_joint_data(0, device, False, 'full_dataset')

# Load the model

In [4]:
def load_config_file():
    with open('model_config_files/Sagittarius_config.json', 'r') as f:
        return json.load(f)

In [5]:
# Now, conduct extrapolation experiment task
initialize_experiment.initialize_random_seed(0)

# Train the model
D = len(dl.get_drug_list())
C = len(dl.get_cell_list())
M = dl.get_feature_dim()
max_dsg = dl.get_max_dosage()
max_time = dl.get_max_time()

sagittarius_manager = manager_for_sagittarius.Sagittarius_Manager_DataLoader(
    M, 2, [D, C], **load_config_file(), minT=0, maxT=max_dsg, num_cont=2,
    device=device, train_transfer=False, other_minT=[0], other_maxT=[max_time])
sagittarius_manager.train_model(
    dl, reload=True, mfile='trained_models/Sagittarius_full_model.pth')

# Start the extrapolation

### These doses and treatment times can be updates to specific doses/treatment times of interest!

In [7]:
dosages_to_simulate = torch.tensor([
    0, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 
    1.6, 1.7, 1.8, 1.9, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,
    8.5, 9.0, 9.5, 10.0] + np.arange(10, 20.1, 0.5).tolist())
treatment_times_to_simulate = torch.tensor(np.arange(4, 49, 4))
gen_k = 10

In [8]:
dosages_to_simulate.shape, treatment_times_to_simulate.shape

(torch.Size([63]), torch.Size([12]))

In [9]:
cell_lines = dl.get_cell_list()
drugs = dl.get_drug_list()

print(C, 'x', D, '=', C * D)

33 x 775 = 25575


In [10]:
# construct a dictionary of possible sources
drug2idx = dl.train_dataset.drug_id_to_idx_mapping
idx2drug = {drug2idx[dr]: dr for dr in drug2idx}
cell2idx = dl.train_dataset.cell_id_to_idx_mapping
idx2cell = {cell2idx[ce]: ce for ce in cell2idx}

sources = {ce: {} for ce in dl.get_cell_list()}  # cell line -> drug -> details
for split in ['train', 'val', 'test']:  # go through the complete dataset
    for expr, dr, ce, dsg, time, mask in dl.get_data_loader(split):
        for i in range(len(expr)):
            sources[idx2cell[ce[i].item()]].update({
                idx2drug[dr[i].item()]: (expr[i], dr[i], ce[i], dsg[i], time[i], mask[i])})

## Create .h5ad file for each cell line

In [11]:
from tqdm import tqdm
import anndata

if not os.path.exists('../simulated_datasets/LINCS/'):
    os.makedirs('../simulated_datasets/LINCS/')

with open('gene_symbol_ordering.txt', 'r') as f:
    gene_ordering = json.load(f)
        
cl_idx = -1
for cl in tqdm(cell_lines):
    initialize_experiment.initialize_random_seed(0)
    
    cl_idx += 1
    print('...{}/{}'.format(cl_idx, C))
    cl_id = cell2idx[cl]
    cl_tensor_src = torch.tensor([cl_id for _ in range(dl.train_dataset.max_unique_cont)]).to(device)
    cl_tensor_tgt = torch.tensor([cl_id for _ in range(len(dosages_to_simulate))]).to(device)
    
    sim_drugs = []
    sim_doses = []
    sim_times = []
    sim_expr = []

    with tqdm(total=D, position=0, leave=True) as pbar:
        for dr in drugs:
            pbar.update()
            dr_id = drug2idx[dr]
            dr_tensor_tgt = torch.tensor([dr_id for _ in range(len(dosages_to_simulate))]).to(device)

            if dr in sources[cl]:  # use this sequence as our starting point
                src_dr = torch.tensor([dr_id for _ in range(dl.train_dataset.max_unique_cont)]).to(device)
                drug_key = dr
            else:  # pick a random drug
                drug_key = np.random.choice(sorted(sources[cl].keys()))
                src_dr = torch.tensor([drug2idx[drug_key] for _ in range(dl.train_dataset.max_unique_cont)]).to(device)
            src_expr = sources[cl][drug_key][0]
            src_dsg = sources[cl][drug_key][3]
            src_time = sources[cl][drug_key][4]
            src_mask = sources[cl][drug_key][5]

            for treatment_time in treatment_times_to_simulate:
                tgt_ttime = torch.tensor([treatment_time for _ in range(len(dosages_to_simulate))]).to(device)

                gen, _, _ = sagittarius_manager.model.generate(
                    src_expr.unsqueeze(0).float(), src_dsg.unsqueeze(0).float(),
                    dosages_to_simulate.unsqueeze(0).float().to(device),
                    [src_dr.unsqueeze(0), cl_tensor_src.unsqueeze(0)], 
                    [dr_tensor_tgt.unsqueeze(0), cl_tensor_tgt.unsqueeze(0)],
                    old_other_ts=[src_time.unsqueeze(0).float()],
                    new_other_ts=[tgt_ttime.unsqueeze(0).float()],
                    old_mask=src_mask.unsqueeze(0), k=gen_k)

                sim_drugs.extend([dr for _ in range(len(dosages_to_simulate))])
                sim_doses.extend([d.item() for d in dosages_to_simulate])
                sim_times.extend(treatment_time for _ in range(len(dosages_to_simulate)))
                sim_expr.append(gen[0].detach().cpu().numpy())  # T' x M
    
    x = np.concatenate(sim_expr)
    obsm = {'drugs': np.asarray(sim_drugs),
            'doses': np.asarray(sim_doses),
            'times': np.asarray(sim_times)}
    var = pd.DataFrame.from_dict({'gene': gene_ordering})
    
    adata = anndata.AnnData(X=x, obsm=obsm, var=var)
    adata.write('../simulated_datasets/LINCS/simulated_{}.h5ad'.format(cl))

  0%|                                                                     | 0/33 [00:00<?, ?it/s]

...0/33


  0%|                                                                    | 0/775 [00:00<?, ?it/s]/homes/gws/addiewc/anaconda3/envs/codeGen/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████████████████████████████████████████████████████| 775/775 [00:52<00:00, 14.89it/s]
/homes/gws/addiewc/anaconda3/envs/codeGen/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
  3%|█▊                                                           | 1/33 [01:07<35:47, 67.11s/it]

...1/33


  6%|███▋                                                         | 2/33 [02:00<30:23, 58.81s/it]

...

 94%|████████████████████████████████████████████████████████▎   | 31/33 [27:44<01:46, 53.05s/it]

...31/33


 97%|██████████████████████████████████████████████████████████▏ | 32/33 [28:37<00:53, 53.21s/it]

...32/33


100%|████████████████████████████████████████████████████████████| 33/33 [29:31<00:00, 53.68s/it]
